# Capstone project Segmenting and Clustering Neighborhoods in Toronto 1st part

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import csv

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())

In [3]:
ttable = soup.find('table',{'class':'wikitable sortable'})
#print(ttable.prettify())

In [4]:
rtab = ttable.findAll('tr')

res = []
for tr in rtab:
    td = tr.findAll('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
        
df_rdata = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])

In [5]:
df_rdata.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
cdf = df_rdata[df_rdata.Borough != 'Not assigned']

In [7]:
cdf.to_csv('cdf.csv', index=False)

In [8]:
cdf_data = pd.read_csv('cdf.csv')

In [9]:
cdf_data.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [10]:
cdf_data['Neighborhood'].replace({'Not assigned':"Queen's Park"}, inplace=True)

In [11]:
cdf_data.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [12]:
cdf_data.to_csv('cdata.csv', index=False)

In [13]:
df_cdata = pd.read_csv('cdata.csv')

In [14]:
dfb= df_cdata.groupby(['PostalCode','Borough'])['Neighborhood']
dfb.apply(', '.join).reset_index()[['PostalCode','Borough','Neighborhood']].to_csv('dfb.csv', index=False)

In [15]:
dfb_data = pd.read_csv('dfb.csv')
dfb_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
geo = 'https://cocl.us/Geospatial_data'
dfgeo = pd.read_csv(geo)
dfgeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
dfgeo.rename(columns=({'Postal Code':'PostalCode'}), inplace=True)
dfgeo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
dftor_data = pd.merge(dfb_data,
                     dfgeo[['PostalCode','Latitude','Longitude']],
                      on='PostalCode')
dftor_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
dftor_data.to_csv('cdftor.csv', index=False)

In [20]:
cdftor_data = pd.read_csv('cdftor.csv')
cdftor_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
cdftor_data.shape

(103, 5)

In [22]:
toront_data = cdftor_data[cdftor_data['Borough'].str.contains('Toronto', regex=False)].reset_index(drop=True)
toront_data.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


### Thank you for reveiwing the first part